# Get Pollo Tropical locations

#### Load Python tools and Jupyter config

In [1]:
import us
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
place = "pollo-tropical"
place_formal = "Pollo Tropical"
color = "#547352"
today = pd.Timestamp.today().strftime("%Y-%m-%d")

---

## Scrape

#### Headers for request

In [4]:
headers = {
    "accept": "application/json, */*",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "x-olo-request": "1",
}

#### Request locations from the API. Apparently they are [only in Florida](https://olo.pollotropical.com/locations). 

In [5]:
response = requests.get(
    "https://olo.pollotropical.com/api/vendors/search/FL", headers=headers
)

#### That returns a list of location dictionaries. Get them into a dataframe.

In [6]:
src = pd.DataFrame(response.json()["vendor-search-results"])[
    ["id", "name", "slug", "phoneNumber", "address", "latitude", "longitude"]
]

#### Deal with nested addresses

In [7]:
src[["street", "city", "zip", "drop", "drop", "country", "state"]] = pd.json_normalize(
    src["address"]
)

#### Format phone numbers

In [8]:
src["phone"] = (
    src["phoneNumber"]
    .str.lstrip("1")
    .str.replace(r"(\d{3})(\d{3})(\d{4})", r"(\1) \2-\3", regex=True)
)

#### Just the columns we need in the order we want

In [9]:
df = src[
    [
        "name",
        "street",
        "city",
        "state",
        "zip",
        "latitude",
        "longitude",
        "phone",
        "slug",
    ]
].copy()

#### The result

In [10]:
df.head()

,name,street,city,state,zip,latitude,longitude,phone,slug
0,PT #10091,3224 East Colonial Dr,Orlando,FL,32803,28.552990,-81.342563,(407) 228-2242,pollo-tropical-e-colonial-drorlando-fl
1,PT #10092,2611 Okeechobee Blvd.,W. Palm Beach,FL,33409,26.707013,-80.092820,(561) 688-0578,pollo-tropical-okeechobee-blvdw-palm-bch-fl
2,PT #10098,3777 East Tamiami Trail,Naples,FL,34112,26.120986,-81.759582,(239) 417-2708,pollo-tropical-east-tamiami-trailnaples-fl
3,PT #10099,8096 West McNab Rd,N Lauderdale,FL,33068,26.207932,-80.235922,(954) 721-2497,pollo-tropical-west-mcnab-rdn-lauderdale-fl
4,PT #10104,4863 Gate Parkway,Jacksonville,FL,32246,30.253688,-81.535537,(904) 646-9707,pollo-tropical-gate-parkwayjacksonville-fl


#### How many locations? 

In [11]:
len(df)

126

#### Create a mapping of state abbreviations to full state names using the us library

In [12]:
state_mapping = {state.abbr: state.name for state in us.states.STATES}

#### New column of full state names based on abbreviations

In [13]:
df["state_name"] = df["state"].map(state_mapping)

#### Make sure our brand name gets in the dataframe

In [14]:
df["brand"] = place_formal

#### Add fetch date

In [15]:
df["updated"] = today

---

## Geography

#### Make it a geodataframe

In [16]:
df_geo = df.copy()

In [17]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

---

## Maps

#### US states background

In [18]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [19]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=5, color=color)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [20]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value(color),
        tooltip=["state:N", "count:Q"],
    )
    .properties(
        title=f"Number of {place_formal} in US, by average lon/lat of locations"
    )
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [21]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [22]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [23]:
gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)